In [17]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
from keras.datasets import mnist 

In [18]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [19]:
test_y 

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [20]:
#Perform one-hot encoding
train_Y = np.zeros((train_y.max()+1, train_y.shape[0])) 
train_Y[train_y, np.arange(train_y.shape[0])] = 1
test_Y = np.zeros((test_y.max()+1, len(test_y)))
test_Y[test_y, np.arange(len(test_y))]=1 
np.shape(train_Y)

(10, 60000)

In [ ]:
class Perceptron:
    
    def __init__(self, max_iters=100, eta=0.5):
        self.max_iters = max_iters 
        self.eta = eta 

    def set_trainData(self, X_train, y_train): 
        self.X_ = X_train 
        d = np.copy(y_train)
        d[d<=0] = 0 
        self.y_ = d
        return self 
    
    def fit(self):
        self.w_ = np.random.rand(1 + self.X_.shape[1])
        self.errors_ = []

        for _ in range(self.max_iters):
            error = 0
            delta_wj = np.copy(self.w_)
            
            # Shuffle the training data
            indices = np.arange(len(self.X_))
            np.random.shuffle(indices)
            X_shuffled = self.X_[indices]
            y_shuffled = self.y_[indices]
            
            for xj, di in zip(X_shuffled, y_shuffled):
                update = self.eta * (di - self.predict(xj))
                self.w_[0] += update 
                self.w_[1:] += update * xj 
                error += np.abs(update / self.eta)
            self.errors_.append(error)

        return self

    def weights(self):
        return self.w_ 
    
    def errors(self):
        return self.errors_ 
    
    def predict(self, X):
        a=np.dot(X, self.w_[1:]) + self.w_[0] 
        return np.where(a >= 0, 1, 0) 




In [29]:
class Network: 

    def __init__(self, layers=5, neuron=5):
        self.layers = layers 
        self.neuron = neuron 

    def set_train(self, train_data, targets):
        self.x_train, self.y_train = train_data[0], train_data[1]
        self.train_data =  [(self.x_train[i,:,:], self.y_train[:,i]) for i in range(np.shape(self.x_train)[0])] 

        # Initialize random weights. Returns a list containing: 
        # Mx(neuron) 2D array in the first index, where M is the number of initial neurons, and (neuron)xN, where N is the number of neurons in the output layer, and (neuron)x(neuron) in the middle layers.
        self.w_ = [np.random.rand(self.neuron, self.neuron) for i in range(self.layers-1)]
        input_w, output_w = np.random.rand(self.neuron, len(self.x_train[0].flatten())), np.random.rand(targets, self.neuron)
        self.w_.insert(0, input_w)
        self.w_.append(output_w)

        # Initialize random column bias vectors. Returns a list containing: 
        # 1xM row vector in the first index, where M is the number of initial neurons, and 1xN, where N is the number of neurons in the output layer, and 1x(neuron) in the middle layers.
        self.bias_ = [np.random.rand(self.neuron, 1) for i in range(self.layers)]
        input_b, output_b = np.zeros((len(self.x_train[0].flatten()),1) ), np.random.rand(targets,1)
        self.bias_.insert(0, input_b)
        self.bias_.append(output_b)

        return self
    
    def feedforward(self, input):
        input = np.column_stack([np.array(input)])
        initial_act=  input + np.array(self.bias_[0])
        self.activations_ = [initial_act] 
        self.z_ = [np.array(input)]
        for l in range(0,self.layers+1): #Loop over l = 0,1,2,...,L-1 where L-1 is the output layer 
            z = np.matmul(self.w_[l], np.column_stack([np.array(self.activations_[l])])) + self.bias_[l+1]
            self.z_.append(z)
            self.activations_.append(self.threshold(z))
        return self
    
    def output_error(self, output):
        err = (self.activations_[-1] - np.column_stack([output]))*self.threshold_der(self.z_[-1])
        return err 
    
    def cost(self, output):
        return (0.5)*np.linalg.norm(self.activations_[-1] - output)**2
    
    def backpropagate(self, output):
        self.error_array = [self.output_error(output)]
        for l in range(-1, -self.layers-1, -1):
            error_vector = np.dot(np.transpose(np.array(self.w_[l])), np.array(self.error_array[l]))*np.column_stack([np.array(self.threshold_der(self.z_[l-1]))])
            self.error_array.insert(0, error_vector)
        return self
    
    def threshold(self, z): #Hyperbolic tangent
        return np.tanh(z)

    def threshold_der(self, z): #Derivative of the activation function 
        return 1 + self.threshold(z)**2

    
    def train(self, test, epochs = 10, eta = 0.01, batch_size=100, ): 
        for iter in range(epochs+1): 
            if test != None:
                print('Epoch {a}/{b}: {c}'.format(a=iter, b= epochs, c=self.evaluate(test)))
            np.random.shuffle(self.train_data) 
            mini_batches = [self.train_data[k:k+batch_size] for k in range(0,len(self.train_data), batch_size)]
            for mini_batch in mini_batches:
                for (m,n) in mini_batch:
                    self.feedforward(np.array(m.flatten()))
                    self.backpropagate(n) 
                    for l in range(-1, -self.layers-2, -1): 
                        update_w = np.dot(self.error_array[l], self.activations_[l-1].T)
                        update_b = self.error_array[l] 
                        self.w_[l] -= update_w*(eta/batch_size)
                        self.bias_[l] -= update_b*(eta/batch_size)
        return self.w_, self.bias_
    
    def evaluate(self, test_data): 
        i = 0
        for (input,output) in test_data:
            input = np.array(input).flatten()
            input = np.column_stack([np.array(input)])
            a_list = [input] 
            z_list = [input]
            for l in range(0,self.layers+1):
                z = np.matmul(self.w_[l], np.column_stack([np.array(a_list[l])])) + self.bias_[l+1]
                z_list.append(z)
                a_list.append(self.threshold(z))
            if np.argmax(a_list[-1]) == output:
                i += 1
        return i/len(test_data)
            
        


In [36]:
import random

# Third-party libraries
import numpy as np

class Network1(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [34]:
net1 = Network(1, 100)
net1.set_train((train_X, train_Y), 10) 
test_data = (test_X, test_y)
test_data_tuple= [(test_X[i], test_y[i]) for i in range(len(test_data[0]))]
net1.train(test=test_data_tuple, epochs=20, eta=0.1, batch_size=100)


Epoch 0/20: 0.098
Epoch 1/20: 0.8314
Epoch 2/20: 0.8467
Epoch 3/20: 0.8446
Epoch 4/20: 0.8421
Epoch 5/20: 0.8425
Epoch 6/20: 0.8381
Epoch 7/20: 0.8405
Epoch 8/20: 0.8542
Epoch 9/20: 0.838
Epoch 10/20: 0.8372
Epoch 11/20: 0.8359
Epoch 12/20: 0.8406
Epoch 13/20: 0.8411
Epoch 14/20: 0.844
Epoch 15/20: 0.8444
Epoch 16/20: 0.8446
Epoch 17/20: 0.8457
Epoch 18/20: 0.8275
Epoch 19/20: 0.8253


([array([[0.28932597, 0.01276585, 0.40338334, ..., 0.08143744, 0.24012045,
          0.26617638],
         [0.20155605, 0.81251396, 0.00322621, ..., 0.96390381, 0.82542192,
          0.60490117],
         [0.75238705, 0.72394835, 0.01593775, ..., 0.49553607, 0.76419978,
          0.15996469],
         ...,
         [0.3653103 , 0.41014683, 0.88756638, ..., 0.76303681, 0.2514801 ,
          0.71449704],
         [0.31408989, 0.2042067 , 0.21751345, ..., 0.6170919 , 0.98864954,
          0.40543242],
         [0.45559055, 0.47320946, 0.33637646, ..., 0.16282167, 0.19293314,
          0.71000183]]),
  array([[-6.61487050e-03,  7.32242711e-03,  1.47918500e-02,
           8.68945314e-03, -5.95528803e-03, -1.18966806e-02,
          -6.98800391e-03, -1.30561464e-02,  5.97854378e-03,
          -2.99094562e-02, -5.23451157e-03,  3.88600554e-03,
          -2.12643199e-02, -1.14814053e-02, -1.93727294e-02,
           1.01775157e-02, -3.42635953e-03,  8.47934333e-03,
           6.22311328e-03,  3.

In [214]:

net1.evaluate(test_data_tuple)

Accuracy: 0.7142
